In [79]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from pathlib import Path
import datetime

# MapBox token
token = "pk.eyJ1IjoiZnlubndpIiwiYSI6ImNrODk3YmF6MzAzcDczbWs5NXdhaGpyNzYifQ.vHweJb-1hjDeE21tTs7tGQ"

## Load data

In [2]:
DIR_base = Path.cwd().parent
DIR_data = Path.joinpath(DIR_base, "data")

In [3]:
data = pd.read_pickle(Path.joinpath(DIR_data, "processed", "temp_yearly_summary.pkl"))
stations = pd.read_pickle(Path.joinpath(DIR_data, "processed", "stations_hourly_temp_hist.pkl"))

## Extract data

In [4]:
lats = stations['latitude']
lons = stations['longitude']
ids = stations['station_id']
names = stations['name']
states = stations['state']
altitudes = stations['altitude']

In [32]:
stations

,station_id,start_date,end_date,altitude,latitude,longitude,name,state
0,3,1950-04-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
1,44,2007-04-01,2020-03-28,44,52.9336,8.2370,Großenkneten,Niedersachsen
2,52,1976-01-01,1988-01-01,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schleswig-Holstein
3,71,2009-12-01,2020-01-02,759,48.2156,8.9784,Albstadt-Badkap,Baden-Württemberg
4,73,2007-04-01,2020-03-28,340,48.6159,13.0506,Aldersbach-Kriestorf,Bayern
...,...,...,...,...,...,...,...,...
653,14138,2009-09-15,2015-12-31,73,52.1655,14.1224,Falkenberg (Grenzschichtmessfeld),Brandenburg
654,15000,2011-04-01,2020-03-28,231,50.7983,6.0244,Aachen-Orsbach,Nordrhein-Westfalen
655,15207,2013-11-01,2020-03-28,317,51.2835,9.3590,Schauenburg-Elgershausen,Hessen
656,15444,2014-09-01,2020-03-28,593,48.4418,9.9216,Ulm-Mähringen,Baden-Württemberg


In [50]:
# store everything in a dict of dataframes, with STATION_ID as key
frames = {}

for station, frame in data.groupby(level=0):
    frames[station] = frame['temperature'].reset_index()[['date', 'max']]
    
frames[44]

,date,max
0,2007-12-31,30.9
1,2008-12-31,32.2
2,2009-12-31,35.0
3,2010-12-31,33.6
4,2011-12-31,30.7
5,2012-12-31,37.0
6,2013-12-31,34.2
7,2014-12-31,32.2
8,2015-12-31,35.2
9,2016-12-31,32.2


In [150]:
start_year = int(data.reset_index().sort_values('date').min()['date'].dt.year)
end_year = int(data.reset_index().sort_values('date').max()['date'].dt.year)

df = data.reset_index()
plotdata = {}

year = start_year
while year <= end_year:
    # search data for particular year
    plotdata[year] = {}

    temp = df.loc[df['date'].dt.year == year]
    idx_list = temp['station_id'].to_list()
    temp_list = temp['temperature']['max'].to_list()

    lon_list = []
    lat_list = []
    name_list = []
    altitude_list = []

    # associate with lon and lat of the station
    for station_id in idx_list:
        row = stations.loc[stations['station_id'] == station_id]
        lon_list.append(float(row['longitude']))
        lat_list.append(float(row['latitude']))
        altitude_list.append(int(row['altitude']))
        name_list.append(row['name'].values[0])
    
    plotdata[year]['lon'] = lon_list
    plotdata[year]['lat'] = lat_list
    plotdata[year]['name'] = name_list
    plotdata[year]['id'] = idx_list
    plotdata[year]['altitude'] = altitude_list
    plotdata[year]['temp'] = temp_list
    
    year += 1
    

## Assemble Plot

In [178]:
# figure
fig = go.Figure()

# mapbox
my_mapbox = {
    'accesstoken': token,
    'style': 'basic',
    'zoom': 5,
    'center': {
        'lon': 10.02,
        'lat': 51.13
    }
}

# layout
fig.update_layout(
    title="DWD Stations in Germany",
    mapbox=my_mapbox,
    autosize=True,
    coloraxis_showscale=True
)



# Add traces, one for each slider step

for k, v in plotdata.items():
    fig.add_trace(
        go.Scattermapbox(
            visible=False,
            mode='markers',
            lat=v['lat'],
            lon=v['lon'],
            marker=go.scattermapbox.Marker(
                size=15,
                color=v['temp'],
                colorscale='thermal',
                cmin=0,
                cmax=45,
                opacity=0.8),
            text=v['temp'],
            
        ))
fig.data[10].visible = True



# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
        label= i + 1893
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Year: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()
fig.write_html("test.html")